# SR basada de Ontologia - Aprendizaje profundo 
El SR basado en Tourismo basado en Ontologia  se encuentra implementado en otro lenguaje y se abstrae los datos a traves de archivos .csv donde se guardan los datosluego son usados en esta implementacion.



In [84]:

import numpy as np
import pandas as pd
import os 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from tensorflow import keras
from tensorflow.keras import layers



# Preprocesamiento

In [85]:
def read_dataset(path = './model_context_pref.csv'):
    dataset = pd.read_csv(path)
    return dataset
def preprocesing(dataset):
    
    user_id = dataset.iloc[:, 1:3].values
    X = dataset.iloc[:, 3:5].values
    Y = dataset.iloc[:, 5].values

    return X,Y,user_id


# Arquitectura de Sistema de recomendacion 

In [86]:
def dnn_etapa2(array_hidden_layers,input_,output): #selected the [128→64→32→16→8] hidden layers
    model = keras.Sequential() #model.add(Embedding(4, 8, input_length=32))
    model.add(layers.Dense(units = array_hidden_layers[0], kernel_initializer = 'uniform', 
                           activation = 'relu', input_dim = input_))

    for idx in range(1,len(array_hidden_layers)):
        model.add(layers.Dense(units = array_hidden_layers[idx] , kernel_initializer = 'uniform', 
                               activation = 'relu'))

    model.add(layers.Dense(units = output, kernel_initializer = 'uniform', activation = 'relu'))

    opt = keras.optimizers.Adam(learning_rate= 0.001)
    model.compile(loss='mean_squared_error', optimizer=opt) #metrics = ['accuracy']

    print(model.summary())
    
    return model
     

# Recomendation


In [87]:
def recomendaciones(y_true,y_pred,user_id_test):
    dict_places = {'1p':0, '2p':1, '3p':2, '4p':3, '5p':4, '6p':5,'7p':6, '8p':7,'9p':8,'10p':9, 
                    '11p':10, '12p':11, '13p':12, '14p':13, '15p':14,'16p':15, '17p':16, '18p':17, '19p':18,'20p':19}
    #dict_places = {'p1':0, 'p2':1, 'p3':2, 'p4':3, 'p5':4, 'p6':5,'p7':6, 'p8':7,'p9':8,'p10':9,
    #               'p11':10, 'p12':11, 'p13':12, 'p14':13, 'p15':14,'p16':15, 'p17':16, 'p18':17, 'p19':18, 'p20':19}
    
    N=7
    matriz_pred = np.zeros((19, 15))
    matriz_true = np.zeros((19, 15))

    for i in range(0,y_pred.shape[0]):
     
        score = y_pred[i][0]  #Se calculo 
        user  = user_id_test[i][0] #id de usuario 
        place  = dict_places[user_id_test[i][1]] #id del place 
        
        #print(user)
        #print(place)
        #print(score)
        if type(user)==str:
            user = i

        matriz_pred[user][place] = score
        matriz_true[user][place] = y_true[i]
    
    data_pred = pd.DataFrame(matriz_pred, columns = dict_places.keys())
    data_dict_pred = data_pred.to_dict('records')
    
    data_true = pd.DataFrame(matriz_true, columns = dict_places.keys())
    data_dict_true = data_true.to_dict('records')

    array_pred = []
    array_true = []

    for idx in range(0,len(data_dict_true)):
        array1 = data_dict_true[idx]
        array2 = data_dict_pred[idx]
       
        pred_ = sorted(array2.items(), key=lambda item: item[1], reverse=True)
        true_ = sorted(array1.items(), key=lambda item: item[1], reverse=True)

        array_pred.append(pred_[0:N])
        array_true.append(true_[0:N])

    return array_pred,array_true


# Metricas


In [88]:
#----------METRICAS----------
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import average_precision_score,precision_score

In [89]:
def rel(n):
    if n>0.5:
        return 1
    else:
        return 0
def precision(y_true,y_pred):

    result = y_true-y_pred
    print(result)

def mean_average_precision(y_pred,y_true):

    MAP = []
    N_total = len(y_pred)
    sumatoria_gen = 0.0

    for idx in  range(0,len(y_pred)):
        k = 7

        usuario_test = y_true[idx]
        alfa = 1
        array_7_res_true = {}
        for i in range(0,k):
            array_7_res_true[usuario_test[i][0]] = alfa
            alfa = alfa - 0.14

        usuario = y_pred[idx]
        array_7_res_pred = []

        for i in range(0,k):
            array_7_res_pred.append(usuario[i][0])
        
        sumatoria=0.0
        for place in array_7_res_pred:
            if place in array_7_res_true:
                sumatoria = sumatoria +1*array_7_res_true[place]
        MAP.append(sumatoria/k)
        sumatoria_gen+=sumatoria/k
    
    MAP_average = sumatoria_gen/N_total

    return MAP_average

In [90]:

def mean_reciprocal_rank(y_pred,y_true):
    sum_total = []
    for idx,value in enumerate(y_true):

        #Por usuario 
        array_true = {}
        idx = 0
        for indx, value in enumerate(value):
            place = value[0]
            array_true[place] = indx

        cont = 0
        sumatoria = 0
        for id_,value in enumerate(y_pred[idx],1):
            place_ = value[0]
            if cont<1:
                if place_ in  array_true and cont < 1 : 
                    sumatoria+= 1/id_
                    cont+=1
            else:
                break
        sum_total.append(sumatoria)
    #print(sum_total)
    return (sum(sum_total)/len(sum_total))



In [91]:
def precision_sr(y_pred,y_true):
    sum_total = []
    for idx,value in enumerate(y_true):

        #Por usuario 
        array_true = {}
        idx = 0
        for indx, value in enumerate(value,1):
            place = value[0]
            if indx<8:
                array_true[place] = indx

        cont = 0
        precision = 0
        intersection = 0
        for id_,value in enumerate(y_pred[idx],1):
            place_ = value[0]
            if id_<8:
                if place_ in  array_true : 
                    intersection +=1 
                cont+=1

        precision+= (intersection/cont)
            
        sum_total.append(precision)
    #print(sum_total)
    return (sum(sum_total)/len(sum_total))

# Llamada a la arquitectura 


In [1]:
array_hidden_layers = [128,256,512] #64,128 #,32,64[8,16,32,64] 
batch_size = 512#512-256
epochs = 50
output = 1


In [2]:
#Cargar datos de entrenamiento 
    
path_train = '../mod_recomendacion/datasetPropuesta/train.csv'
dataset  = read_dataset(path_train)

#print("Carga de datos")
#print(dataset)
#print("\n")
X_train,y_train,user_id = preprocesing(dataset) #X y Y entrenamiento 
#print("X_train ->",X_train)
#print("Y_train ->",y_train)
#print("user_id",user_id)

#Cargar datos de test
path_test = '../mod_recomendacion/datasetPropuesta/test_individual.csv'
dataset_test  = read_dataset(path_test)
X_test,y_test,user_id_test = preprocesing(dataset_test) #X y Y entrenamiento 
    
model = dnn_etapa2(array_hidden_layers, X_train.shape[1], output)
model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs)
y_pred = model.predict(X_test)
print('y_pred : ',y_pred)

NameError: name 'read_dataset' is not defined

In [94]:
#array_pred,array_true = recomendaciones(y_test,y_pred,user_id_test)

#array_pred
"""predic_recomendation = {}
idx_= 0
for idx in array_pred:
    predic_recomendation[idx_] = []
    for colidx in idx:
        predic_recomendation[idx_].append(colidx[0])
    idx_+=1
data_pref_pd = pd.DataFrame.from_dict(predic_recomendation, orient='index')"""
#data_pref_pd


"predic_recomendation = {}\nidx_= 0\nfor idx in array_pred:\n    predic_recomendation[idx_] = []\n    for colidx in idx:\n        predic_recomendation[idx_].append(colidx[0])\n    idx_+=1\ndata_pref_pd = pd.DataFrame.from_dict(predic_recomendation, orient='index')"

In [97]:
map_ = mean_average_precision(array_pred,array_true)
mrk = mean_reciprocal_rank(array_pred,array_true)
precision = precision_sr(array_pred,array_true)
    
    
print("-----------------------------------------------")
print("--------------------METRICAS ------------------")
print("-----------------------------------------------")

mae  =  mean_absolute_error(y_test,y_pred)
    # Calculation of Mean Squared Error (MSE)
mse = mean_squared_error(y_test,y_pred)
mape = mean_absolute_percentage_error(y_pred,y_test)

    
print(" MAE       : ", round(mae,3))
print(" MSE       : ", round(mse,3))
print(" MAPE      : ", round(mape,3))
print(" MAP       : ", round(map_,3))
print(" MRK       : ", round(mrk,3))
print(" Precision : ", round(precision,3))

-----------------------------------------------
--------------------METRICAS ------------------
-----------------------------------------------
 MAE       :  0.114
 MSE       :  0.014
 MAPE      :  0.157
 MAP       :  0.325
 MRK       :  0.796
 Precision :  0.511


In [12]:
array_hidden_layers = [64,128,256,512] #64,128 #,32,64[8,16,32,64] 
batch_size = 512#512-256
epochs = 50
output = 1


